In [173]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt 
import re 

import scipy.stats as stats  

%matplotlib inline 

sns.set(style = "ticks")

from dateutil.relativedelta import relativedelta
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [174]:
df = pd.read_excel('ads_data.xlsx')

In [175]:
df1 = df.copy()

# ***Simple Linear Regression***

In [176]:
df1['Total_Marketing_spend'] = df1['TV'] + df1['Radio'] + df1['Newspaper']

In [177]:
df1.head(2)

,S_no,TV,Radio,Newspaper,Sales,Total_Marketing_spend
0,1,230.1,37.8,69.2,22.1,337.1
1,2,44.5,39.3,45.1,10.4,128.9


In [178]:
df1 = df1.loc[ : , ['Total_Marketing_spend', 'Sales']]

In [179]:
df1.head(2)

,Total_Marketing_spend,Sales
0,337.1,22.1
1,128.9,10.4


In [180]:
# importing module

In [181]:
ols_model = smf.ols('Sales~Total_Marketing_spend', df1).fit()

In [182]:
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     603.4
Date:                Fri, 15 Nov 2024   Prob (F-statistic):           5.06e-62
Time:                        13:27:21   Log-Likelihood:                -473.88
No. Observations:                 200   AIC:                             951.8
Df Residuals:                     198   BIC:                             958.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 4.24

In [183]:
df1['pred_sales'] = 0.0487*df1['Total_Marketing_spend'] + 4.2430

In [184]:
df1.head(2)

,Total_Marketing_spend,Sales,pred_sales
0,337.1,22.1,20.65977
1,128.9,10.4,10.52043


In [185]:
df1['absolute_error'] = np.abs(df1['Sales'] - df1['pred_sales'])

In [186]:
df1.head(2)

,Total_Marketing_spend,Sales,pred_sales,absolute_error
0,337.1,22.1,20.65977,1.44023
1,128.9,10.4,10.52043,0.12043


In [187]:
df1['absolute_percent_error'] = (df1['absolute_error']/df1['Sales'])*100

In [188]:
df1.head(2)

,Total_Marketing_spend,Sales,pred_sales,absolute_error,absolute_percent_error
0,337.1,22.1,20.65977,1.44023,6.516878
1,128.9,10.4,10.52043,0.12043,1.157981


In [189]:
MAPE = np.mean(df1['absolute_percent_error']).round(2)

In [190]:
MAPE 

15.87

In [191]:
x=df1.drop("absolute_percent_error",axis=1)
y=df1.absolute_percent_error

In [192]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=51)

In [193]:
linear_reg = LinearRegression()

In [194]:
linear_reg.fit(x_train,y_train)

LinearRegression()

In [195]:
linear_reg.coef_

array([ 3.80562487e-02, -2.62771951e+00,  1.85333931e-03,  1.02145037e+01])

In [196]:
linear_reg.intercept_

25.567371246769724

In [197]:
pred=linear_reg.predict(x_test)

In [198]:
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r2)

Mean Absolute Error (MAE): 6.048367941804449
Mean Squared Error (MSE): 61.48574830824748
Root Mean Squared Error (RMSE): 7.841284863352911
R-squared (R²): 0.6457730446716214


##  ***Multivariate Regression***

In [199]:
df = df.drop (columns = 'S_no')

In [200]:
df.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [201]:
ols_model2 = smf.ols('Sales ~ TV + Radio', df).fit()

In [202]:
print(ols_model2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     859.6
Date:                Fri, 15 Nov 2024   Prob (F-statistic):           4.83e-98
Time:                        13:27:21   Log-Likelihood:                -386.20
No. Observations:                 200   AIC:                             778.4
Df Residuals:                     197   BIC:                             788.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9211      0.294      9.919      0.0

In [203]:
df['Pred_value'] = 0.0458*df['TV'] + 0.1880* df['Radio'] - 0.0010* df['Newspaper'] + 2.9389

In [204]:
df.head(2)

,TV,Radio,Newspaper,Sales,Pred_value
0,230.1,37.8,69.2,22.1,20.51468
1,44.5,39.3,45.1,10.4,12.32030


In [205]:
df['Error'] = df['Sales'] - df['Pred_value']

In [206]:
df['Percent_error'] = round((df['Error']/df['Sales'])*100,2)

In [207]:
df['Absolute_percent_error'] = np.abs(df['Percent_error'])

In [208]:
MAPE = np.mean(df['Absolute_percent_error']).round(2)

In [209]:
MAPE

13.87

In [210]:
mean_value = round(np.mean(df['Sales']),3)

In [211]:
mean_value

14.022

In [212]:
df['basic_deviation'] = df['Sales']-mean_value

In [213]:
df['abs_percent_basic_dev'] = np.abs(round((df['basic_deviation']/df['Sales'])*100,2))

In [214]:
MAPE_basic = round(np.mean(df['abs_percent_basic_dev']),2)

In [215]:
MAPE_basic

40.06

In [216]:
df.head()

,TV,Radio,Newspaper,Sales,Pred_value,Error,Percent_error,Absolute_percent_error,basic_deviation,abs_percent_basic_dev
0,230.1,37.8,69.2,22.1,20.51468,1.58532,7.17,7.17,8.078,36.55
1,44.5,39.3,45.1,10.4,12.32030,-1.92030,-18.46,18.46,-3.622,34.83
2,17.2,45.9,69.3,9.3,12.28656,-2.98656,-32.11,32.11,-4.722,50.77
3,151.5,41.3,58.5,18.5,17.58350,0.91650,4.95,4.95,4.478,24.21
4,180.8,10.8,58.4,12.9,13.19154,-0.29154,-2.26,2.26,-1.122,8.70
